In [1]:
# The libraries we'll use in this report are:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import matplotlib.dates as dt
import matplotlib.lines as mlines
from IPython.core.pylabtools import figsize
import warnings
import pickle

# Processing
from sklearn.preprocessing import StandardScaler,PolynomialFeatures,MinMaxScaler
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.model_selection import train_test_split,KFold,GridSearchCV,RandomizedSearchCV, StratifiedKFold,cross_val_predict,cross_val_score,ShuffleSplit
from sklearn.feature_selection import SelectKBest, SelectFromModel
from imblearn.over_sampling import SMOTE 
from imblearn.under_sampling import NearMiss
from sklearn.compose import ColumnTransformer
from sklearn.base import clone

# Modelling
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression

# Representation Learning for the autoencoder
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Activation, Dense, Dropout
from tensorflow.keras.layers import BatchNormalization, Input, Lambda
from tensorflow.keras import regularizers
from tensorflow.keras.losses import mse, binary_crossentropy

# Evaluation
from sklearn.metrics import roc_curve,auc, roc_auc_score,accuracy_score, recall_score, f1_score, confusion_matrix, log_loss
from sklearn.metrics import classification_report, balanced_accuracy_score,precision_score,average_precision_score, precision_recall_curve

plt.rcParams['figure.figsize'] = (20, 10)
pd.set_option('display.max_columns', None)
warnings.filterwarnings('ignore')
sns.set()

In [2]:
import sys, sklearn
print(f'sklearn    {sklearn.__version__}')
print(f'tensorflow {tf.__version__}')
print(f'keras      {keras.__version__}')
print(f'numpy      {np.__version__}')
print(f'pandas   {pd.__version__}')

sklearn    0.23.1
tensorflow 2.2.0
keras      2.3.0-tf
numpy      1.18.5
pandas   1.0.4


In [3]:
# 
tf.random.set_seed(2020)
np.random.seed(2020)

In [5]:
#devices = pd.read_csv("device_failure.csv", encoding='Latin-1')
filename = 'Folds5x2_pp.xlsx'
df = pd.read_excel(filename, sheet_name='Sheet1')
df.describe()

,AT,V,AP,RH,PE
count,9568.000000,9568.000000,9568.000000,9568.000000,9568.000000
mean,19.651231,54.305804,1013.259078,73.308978,454.365009
std,7.452473,12.707893,5.938784,14.600269,17.066995
min,1.810000,25.360000,992.890000,25.560000,420.260000
25%,13.510000,41.740000,1009.100000,63.327500,439.750000
50%,20.345000,52.080000,1012.940000,74.975000,451.550000
75%,25.720000,66.540000,1017.260000,84.830000,468.430000
max,37.110000,81.560000,1033.300000,100.160000,495.760000


In [6]:
X, Y = df[['AT', 'V','AP','RH']], df['PE']
scaler = MinMaxScaler()
X_new = scaler.fit_transform(X)
target_scaler = MinMaxScaler()
Y_new = target_scaler.fit_transform(Y.values.reshape(-1,1))
X_train, X_test, Y_train, y_test = \
  train_test_split(X_new, Y_new, test_size=0.4, random_state=333)

In [14]:
class LinearRegressor:
    def __init__(self,d, lr=0.001 ):

        self.b = tf.Variable(0.0, dtype=tf.float32)
        self.W = tf.Variable(tf.random.normal([d,1]),\
                             dtype=tf.float32)
            
        # The Linear Regression Model
        self.F = self.function(self)
        
        # Loss function
        self.loss = tf.reduce_mean(tf.square(self.Y - self.F,\
                                        name='LSE'))

        # Gradient Descent with learning 
        # rate of 0.05 to minimize loss
        optimizer = tf.train.GradientDescentOptimizer(lr)
        self.optimize = optimizer.minimize(self.loss)

        # Initializing Variables
        init_op = tf.global_variables_initializer()
        self.sess = tf.Session()
        self.sess.run(init_op)

In [14]:

def function(self, X):
    return tf.matmul(X, self.W) + self.b

def fit(self, X, Y,epochs=500):
    total = []
    for i in range(epochs):
        _, l = self.sess.run([self.optimize,self.loss],\
                        feed_dict={self.X: X, self.Y: Y})
        total.append(l)
        if i%100==0:
            print('Epoch {0}/{1}: Loss {2}'.format(i,epochs,l))
    return total

def predict(self, X):
    return self.sess.run(self.function(X), feed_dict={self.X:X})

def get_weights(self):
    return self.sess.run([self.W, self.b])

In [15]:


N, d = X_train.shape
model = LinearRegressor(d)
#loss = model.fit(X_train, Y_train)

ValueError: Attempt to convert a value (<__main__.LinearRegressor object at 0x7ff8716744e0>) with an unsupported type (<class '__main__.LinearRegressor'>) to a Tensor.

In [9]:
loss = model.fit(X_train, Y_train, 20000)

NameError: name 'model' is not defined

In [ ]:
Y_pred = model.predict(np.float32(X_test))

In [ ]:
print("R2 Score is {} and MSE {}".format(\
       r2_score(y_test, Y_pred),\
       mean_squared_error(y_test, Y_pred)))

In [ ]:
plt.scatter(Y_pred, y_test)
#plt.plot(X_test, Y_pred)
plt.xlabel("Actual Net Hourly Electrical Energy Output (MW)")
plt.ylabel("Predicted Net Hourly Electrical Energy Output (MW)")

In [ ]:
print("The weights and biase are", model.get_weights())